# Inference 환경구성(TF Hub와 Transfer Learning)

## 설치하기


In [0]:
try: 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf 
import matplotlib.pylab as plt

In [0]:
!pip install -U tf-hub-nightly
import tensorflow_hub as hub

from tensorflow.keras import layers

## ImageNet Classifier


### Classifier 다운로드

Transfer Learning을 위한 네트워크 로드를 위해 `hub.module`을, 그리고 하나의 keras 층으로 감싸기 위해 `tf.keras.layers.Lambda`를 사용한다.

TF 2.0에서 사용할 수 있는 Image Classifier는 [이 링크](https://tfhub.dev/s?q=tf2&module-type=image-classification)에서 확인할 수 있다.

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

### Single image에 대해 수행하기


다운로드 받은 TF Hub 모델을 사용하기 위해 단일 이미지를 다운로드 받는다.

In [0]:
import numpy as np
import PIL.Image as Image

dog_img = tf.keras.utils.get_file('image.jpg', 'https://blogfiles.pstatic.net/MjAxOTEwMTFfMzMg/MDAxNTcwNzY3MTI2MTk4.bUgUVq4WmXRSXuqlS5dWLOmuWNN2-OIfqCPNy-dAGEUg.EYmc3-w4lSogPESlHy8u1_8ZBIiPygb2B5HMUqsToIcg.JPEG.hkc0929/%EB%B0%98%EB%A0%A4%EA%B2%AC_%ED%82%A4%EC%9A%B0%EB%A9%B4_%EC%8B%AC%EC%9E%A5%EB%B3%91_%EC%82%AC%EB%A7%9D%EB%A5%A0_30_%EC%A4%84%EC%96%B4%EC%9A%94xxx.jpeg')
dog_img = Image.open(dog_img).resize(IMAGE_SHAPE)
dog_img

In [0]:
dog_img = np.array(dog_img)/255.0
dog_img.shape

Batch 차원을 추가하고, 모델에 통과시킨다.

In [0]:
result = classifier.predict(dog_img[np.newaxis, ...])
result.shape

그 결과는 Logit의 1001 요소 벡터이며, 이는 이미지에 대한 각각의 클래스 확률을 계산한다.

가장 높은 확률의 클래스 ID를 확인할 수 있다.

In [0]:
predicted_class = np.argmax(result[0], axis=-1)
predicted_class

### 예측 해독하기

클래스 ID를 예측하고 `ImageNet`라벨을 불러온 뒤 예측을 해독한다.

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
print(imagenet_labels[:20])

In [0]:
plt.imshow(dog_img)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## Simple Transfer Learning


TF Hub를 사용하면 사용자 개인의 데이터셋의 클래스를 인식하기 위한 top layer fine tuning을 쉽게 수행할 수 있다.

### Dataset

이 예제를 살펴보기 위해, TF의 flowers 데이터셋을 사용할 것이다.

In [0]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

`tf.keras.preprocessing.image.image.ImageDataGenerator`를 사용하여 데이터를 로드

> label : ['sunflowers', 'daisy', 'roses', 'tulips', 'dandelion']

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
image_data = image_generator.flow_from_directory(str(data_root), batch_size=32, target_size=IMAGE_SHAPE)

In [0]:
plt.imshow(image_data[0][0][0]) 

결과로 나온 `image_data`객체는 `image_batch`와 `label_batch`를 같이 리턴 하는 repeater이다.

In [0]:
for image_batch, label_batch in image_data:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

### Image Batch에 대한 Classifier 실행

이제 이미지 배치에 대한 분류기를 실행해보자.

In [0]:
result_batch = classifier.predict(image_batch)
result_batch.shape

In [0]:
predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
predicted_class_names

얼마나 많은 예측들이 이미지에 잘 들어맞는지 확인해보자.

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

이미지 속성을 가진 `LICENSE.txt` 파일을 확인해보자.

결과가 완벽하진 않지만, 모델이 ("daisy"를 제외한) 모든 것을 대비해서 학습된 클래스가 아니라는 것을 고려하면 꽤나 합리적이다.

하지만 결코 만족스럽진 않다...

### Download Headless Model

Image feature vector를 추출하기 위한 TF Hub 모델은 [이 링크](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2)에서 확인할 수 있다.

In [0]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" 

Feature extractor를 만들어보자.

In [0]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=(224,224,3))

각각의 이미지마다 길이가 1280인 벡터가 반환되는 것을 확인할 수 있다.

In [0]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

Feature extractor layer에 있는 변수들을 학습 불가능하도록 만들면, 학습은 오직 새로운 classifier layer에만 가능하게 된다.

In [0]:
feature_extractor_layer.trainable = False   # tf hub에서 가져온 layer는 기본적으로 trainable=false 옵션이 설정되어있다

### Classifier head를 붙이기

이제 `tf.keras.Sequential` 모델에 있는 허브 layer을 감싸고, 새로운 classifier layer를 추가하자.

In [0]:
model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(image_data.num_classes, activation='softmax')
])

model.summary()

In [0]:
predictions = model(image_batch)

In [0]:
predictions.shape

### 모델 학습

학습과정의 환경을 설정하기 위해 compile한다.

In [0]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['acc'])

2 epoch만 학습시켜보자.

학습과정을 시각화하기 위해 아래 방법을 사용하면 Epoch단위가 아니라 배치 단위의 loss와 accuracy를 기록할 수 있다.

In [0]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [0]:
steps_per_epoch = np.ceil(image_data.samples/image_data.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit(image_data, epochs=2,
                              steps_per_epoch=steps_per_epoch,
                              callbacks = [batch_stats_callback], verbose=1)

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)

In [0]:
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

### Prediction 확인

클래스 이름들을 정렬한다.

In [0]:
class_names = sorted(image_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

모델을 통해 이미지 배치를 실행시킨 뒤 튀어나온 인덱스들을 클래스 이름으로 바꾼다.

In [0]:
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

결과를 예측한다.

In [0]:
true_label_id = np.argmax(label_batch, axis=-1)

In [0]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == true_label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

## Model export하기

학습시킨 모델을 저장해보자.

In [0]:
t

In [0]:
model_name = '20200423_mymodel'

export_path = "/tmp/saved_models/"+model_name
model.save(export_path, save_format='tf')

export_path

Export된 모델을 다시 로딩할 수 있고, 이는 동일한 결과를 도출한다.

In [0]:
reloaded = tf.keras.models.load_model(export_path)

In [0]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [0]:
abs(reloaded_result_batch - result_batch).max()

저장된 모델로부터 추론을 수행할 수도 있고,[TFLite](https://www.tensorflow.org/lite/convert/) 나 [TFjs](https://github.com/tensorflow/tfjs-converter) 로 변환할 수 있다.
